In [1]:
from keras import backend as K
from scipy.ndimage import zoom
import numpy as np
import os
import keras
import tensorflow as tf
import nibabel as nib
import matplotlib.pyplot as plt

In [45]:
def grad_cam(model, img, class_idx, layer_name="conv3d_3"):
    """
    Calcula o Grad-CAM para uma imagem e um índice de classe específicos.
    
    :param model: O modelo Keras.
    :param img: A imagem de entrada (formato (1, altura, largura, canais)).
    :param class_idx: O índice da classe de interesse.
    :param layer_name: O nome da camada convolucional onde o Grad-CAM será calculado.
    
    :return: O mapa Grad-CAM.
    """

    # Garantir que o modelo seja chamado para inicializar as entradas
    img = tf.convert_to_tensor(img)  # Certificar-se de que a imagem é um tensor
    model(img)  # Forçar uma chamada para o modelo para inicializar as entradas

    # Obter a camada convolucional final do modelo
    last_conv_layer = model.get_layer(layer_name)
    
    # Criar um novo modelo que retorna a saída da camada convolucional e a saída final
    grad_model = keras.Model(inputs=model.input, outputs=[last_conv_layer.output, model.output])
    
    # Obter as previsões para a imagem
    with tf.GradientTape() as tape:
        tape.watch(img)  # Habilita a vigilância da imagem (variável de entrada)
        conv_output, class_output = grad_model(img)  # Saídas da camada convolucional e da classificação
        
        # Acessar a classe desejada
        class_loss = class_output[:, class_idx]
    
    # Calcular o gradiente da perda da classe com relação à saída da camada convolucional
    grads = tape.gradient(class_loss, conv_output)
    
    # Calcular a média global do gradiente sobre a dimensão espacial (altura e largura)
    grads = tf.reduce_mean(grads, axis=(1, 2))
    
    # Expandir os gradientes para as dimensões do mapa de ativação
    conv_output = conv_output[0]  # Remover batch dimension
    grad_cam = tf.reduce_sum(grads[..., tf.newaxis] * conv_output, axis=-1)  # Multiplicação ponderada
    
    # Normalizar o Grad-CAM
    grad_cam = np.maximum(grad_cam, 0)  # Retirar valores negativos
    grad_cam = grad_cam / np.max(grad_cam)  # Normalizar entre 0 e 1
    
    # Redimensionar para o tamanho original da imagem
    grad_cam = tf.image.resize(grad_cam, (img.shape[1], img.shape[2]))  # Redimensionar
    grad_cam = grad_cam.numpy()  # Converter para um array numpy
    
    # Exibir o Grad-CAM
    plt.imshow(grad_cam, cmap='jet')
    plt.colorbar()
    plt.show()

    return grad_cam

In [40]:
base_path = "c:/Users/Paulo Pires/Desktop/Alzheimer_cnn/7_slices_axial"
model_path = f"{base_path}/results/3d/test_1/binary_classifier_100_epochs_batch_64_2_classes.keras"
sample_path = f"{base_path}/validation/ad/I89119.nii.gz"

model = tf.keras.models.load_model(model_path)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


sample = nib.load(sample_path).get_fdata()
sample = np.expand_dims(sample, axis=0)  # Adiciona batch size
sample = np.expand_dims(sample, axis=-1)  # Adiciona canal

In [ ]:
print(f"Forma da entrada do modelo: {model.input_shape}")
print(f"Forma do sample: {sample.shape}")


In [ ]:
model.summary()

In [ ]:
_ = model.predict(sample)

In [ ]:
# Escolher o índice da classe para a qual gerar o Grad-CAM (Exemplo: classe 0 ou 1)
class_idx = 1  # Altere para 1 para a outra classe

# Gerar o Grad-CAM
grad_cam_map = grad_cam(model, sample, class_idx)

In [ ]:
# Exibir o Grad-CAM
plt.imshow(grad_cam_map[:, :, 0], cmap='jet')  # Exibindo a primeira camada
plt.colorbar()
plt.show()